In [1]:
# The code was removed by Watson Studio for sharing.

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190915161548-0000
KERNEL_ID = 99f218ca-dd85-4d51-9669-6d58afc8580e


# IEEE CIS Fraud Detection - ETL

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pixiedust.display import *

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table SPARK_PACKAGES created successfully
Table USER_PREFERENCES created successfully
Table service_connections created successfully


## Training Set

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
# We have already ETLed the training set in data exploration
df_train = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('transaction_id_train.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))

## Test Set

In [38]:
test_identity = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('test_identity.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))

In [39]:
test_transaction = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('test_transaction.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))

In [40]:
test_identity.createOrReplaceTempView("test_identity")
test_transaction.createOrReplaceTempView("test_transaction")

### Joining Transaction and Identity

In [41]:
columns = test_identity.columns[1:]
queryCols = ''
for col in columns:
    queryCols += ', b.'+col
queryCols

', b.id_01, b.id_02, b.id_03, b.id_04, b.id_05, b.id_06, b.id_07, b.id_08, b.id_09, b.id_10, b.id_11, b.id_12, b.id_13, b.id_14, b.id_15, b.id_16, b.id_17, b.id_18, b.id_19, b.id_20, b.id_21, b.id_22, b.id_23, b.id_24, b.id_25, b.id_26, b.id_27, b.id_28, b.id_29, b.id_30, b.id_31, b.id_32, b.id_33, b.id_34, b.id_35, b.id_36, b.id_37, b.id_38, b.DeviceType, b.DeviceInfo'

In [44]:
query = f"""
SELECT 
    a.* {queryCols} 
    FROM test_transaction a
    LEFT JOIN test_identity b
    ON a.transactionID = b.transactionID
"""

transaction_identity = sql(query)
transaction_identity.createOrReplaceTempView("transaction_identity")

In [42]:
test_identity.count()

141907

In [43]:
test_transaction.count()

506691

In [45]:
transaction_identity.count()

506691

In [10]:
display(transaction_identity)

TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
3713171,20054054,75.0,R,15592,494.0,150.0,mastercard,102.0,credit,299.0,87.0,None,36.0,yahoo.com,yahoo.com,2.0,1.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,176435.0,None,None,0.0,0.0,None,None,None,None,100.0,Found,52.0,-480.0,New,NotFound,166.0,None,215.0,222.0,None,None,None,None,None,None,None,New,NotFound,Windo

In [11]:
# select only columns in both training and test
col_train = df_train.columns

In [13]:
col_query = col_train[0]
for col in col_train[1:]:
    if col != 'isFraud':
        col_query = col_query + ', ' + col
col_query

'D3, D2, D5, D7, dist2, id_18, D12, D13, D14, V50, V38, V40, V52, V51, V37, V47, V46, V44, V43, V42, V41, V45, V39, D4, id_04, id_03, D6, V56, V55, V57, V64, V58, V71, V60, V61, V62, V63, V72, V59, V68, V65, V74, V73, V67, V66, V92, V85, V93, V94, V88, V87, V86, V78, V84, V83, V82, V80, V79, V77, V81, V89, D15, V34, V22, V33, V14, V15, V16, V17, V18, V19, V20, V21, V23, V24, V25, V26, V27, V28, V32, V31, D10, D9, D8, id_10, id_09, id_32, id_14, V147, V148, V149, V153, V154, V155, V142, V141, V156, V157, V140, V146, V161, V139, V138, V163, V162, V158, V164, V143, V144, V145, V150, V151, V152, V159, V165, V160, V166, V331, V333, V334, V332, V335, V336, V337, V324, V325, V330, V329, V339, V328, V327, V326, V322, V323, V338, addr2, addr1, id_13, V253, V269, V274, V275, V276, V277, V278, V252, V235, V233, V232, V231, V230, V229, V228, V226, V217, V218, V225, V224, V219, V273, V223, V268, V258, V249, V248, V247, V246, V244, V243, V242, V241, V240, V237, V267, V254, V257, V236, V266, V265, V2

In [14]:
query = f"""
SELECT 
    {col_query}
    FROM transaction_identity
"""

transaction_identity2 = sql(query)
transaction_identity2.createOrReplaceTempView("transaction_identity2")

In [15]:
# save dataset
transaction_identity_pd = transaction_identity2.toPandas()

# save file
# Save dataframe as csv file to storage
project.save_data(data=transaction_identity_pd.to_csv(index=False), file_name='transaction_id_test.csv',overwrite=True)

{'file_name': 'transaction_id_test.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'default-donotdelete-pr-l16zufkrkht8vm',
 'asset_id': 'bca9469a-4fe8-4c9a-82ae-18192c64293c'}

## Data Cleansing

In [5]:
df_test = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('transaction_id_test.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))

In [16]:
# df_train loaded already above
df_train.createOrReplaceTempView("df_train")
df_test.createOrReplaceTempView("df_test")

### Data Types

In [13]:
display(df_train)

D3,D2,D5,D7,dist2,id_18,D12,D13,D14,V50,V38,V40,V52,V51,V37,V47,V46,V44,V43,V42,V41,V45,V39,D4,id_04,id_03,D6,V56,V55,V57,V64,V58,V71,V60,V61,V62,V63,V72,V59,V68,V65,V74,V73,V67,V66,V92,V85,V93,V94,V88,V87,V86,V78,V84,V83,V82,V80,V79,V77,V81,V89,D15,V34,V22,V33,V14,V15,V16,V17,V18,V19,V20,V21,V23,V24,V25,V26,V27,V28,V32,V31,D10,D9,D8,id_10,id_09,id_32,id_14,V147,V148,V149,V153,V154,V155,V142,V141,V156,V157,V140,V146,V161,V139,V138,V163,V162,V158,V164,V143,V144,V145,V150,V151,V152,V159,V165,V160,V166,V331,V333,V334,V332,V335,V336,V337,V324,V325,V330,V329,V339,V328,V327,V326,V322,V323,V338,addr2,addr1,id_13,V253,V269,V274,V275,V276,V277,V278,V252,V235,V233,V232,V231,V230,V229,V228,V226,V217,V218,V225,V224,V219,V273,V223,V268,V258,V249,V248,V247,V246,V244,V243,V242,V241,V240,V237,V267,V254,V257,V236,V266,V265,V260,V264,V263,V261,V262,id_06,id_05,id_20,id_19,id_17,V203,V214,V193,V215,V216,V199,V204,V213,V205,V206,V207,V196,V202,V168,V192,V178,V211,V167,V172,V173,V176,V177,V179,V191,V181,V182,V183,V186,V187,V190,V212,V169,V210,V194,V209,V170,V171,V174,V175,V180,V184,V188,V189,V185,V195,V197,V198,V200,V201,V208,id_02,id_11,V251,V238,V239,V234,V256,V255,V227,V222,V250,V221,V220,V270,V245,V271,V272,V259,card5,card2,V296,V314,V313,D1,V300,V301,V288,V315,V281,V282,V283,V289,card3,V104,V105,V95,V96,V97,V98,V100,V101,V102,V103,V99,V116,V106,V123,V137,V136,V135,V134,V133,V132,V131,V130,V129,V128,V127,V126,V125,V124,V122,V107,V121,V120,V119,V118,V117,V115,V114,V113,V112,V111,V110,V109,V108,V320,V321,V291,V319,V279,V298,V294,V293,V292,V299,V290,V287,V318,V286,V302,V285,V295,V280,V284,V307,V303,V317,V316,V312,V311,V310,V309,V308,V297,V306,V305,V304,card1,TransactionAmt,TransactionDT,isFraud,C1,C2,C3,C4,C6,C7,C8,id_01,C10,C11,C12,C13,C14,TransactionID,id_33,id_30,id_34,DeviceInfo,id_16,P_emaildomain,R_emaildomain,id_31,DeviceType,id_29,id_28,id_38,id_35,id_37,id_15,id_36,card4,card6,ProductCD,id_12
0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,-1.0,0.0,507.0,266.0,225.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,185152.0,100.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,226.0,540.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,106.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,11810.0,23.786,1403914.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,-10.0,1.0,1.0,1.0,1.0,1.0,3049200.0,None,None,None,Windows,NotFound,gmail.com,gmail.com,chrome 63.0,desktop,NotFound,New,T,F,T,New,F,visa,debit,C,NotFound
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.791666,0.791666,0.0,0.0,32.0,-300.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.

In [14]:
display(df_test)

D3,D2,D5,D7,dist2,id_18,D12,D13,D14,V50,V38,V40,V52,V51,V37,V47,V46,V44,V43,V42,V41,V45,V39,D4,id_04,id_03,D6,V56,V55,V57,V64,V58,V71,V60,V61,V62,V63,V72,V59,V68,V65,V74,V73,V67,V66,V92,V85,V93,V94,V88,V87,V86,V78,V84,V83,V82,V80,V79,V77,V81,V89,D15,V34,V22,V33,V14,V15,V16,V17,V18,V19,V20,V21,V23,V24,V25,V26,V27,V28,V32,V31,D10,D9,D8,id_10,id_09,id_32,id_14,V147,V148,V149,V153,V154,V155,V142,V141,V156,V157,V140,V146,V161,V139,V138,V163,V162,V158,V164,V143,V144,V145,V150,V151,V152,V159,V165,V160,V166,V331,V333,V334,V332,V335,V336,V337,V324,V325,V330,V329,V339,V328,V327,V326,V322,V323,V338,addr2,addr1,id_13,V253,V269,V274,V275,V276,V277,V278,V252,V235,V233,V232,V231,V230,V229,V228,V226,V217,V218,V225,V224,V219,V273,V223,V268,V258,V249,V248,V247,V246,V244,V243,V242,V241,V240,V237,V267,V254,V257,V236,V266,V265,V260,V264,V263,V261,V262,id_06,id_05,id_20,id_19,id_17,V203,V214,V193,V215,V216,V199,V204,V213,V205,V206,V207,V196,V202,V168,V192,V178,V211,V167,V172,V173,V176,V177,V179,V191,V181,V182,V183,V186,V187,V190,V212,V169,V210,V194,V209,V170,V171,V174,V175,V180,V184,V188,V189,V185,V195,V197,V198,V200,V201,V208,id_02,id_11,V251,V238,V239,V234,V256,V255,V227,V222,V250,V221,V220,V270,V245,V271,V272,V259,card5,card2,V296,V314,V313,D1,V300,V301,V288,V315,V281,V282,V283,V289,card3,V104,V105,V95,V96,V97,V98,V100,V101,V102,V103,V99,V116,V106,V123,V137,V136,V135,V134,V133,V132,V131,V130,V129,V128,V127,V126,V125,V124,V122,V107,V121,V120,V119,V118,V117,V115,V114,V113,V112,V111,V110,V109,V108,V320,V321,V291,V319,V279,V298,V294,V293,V292,V299,V290,V287,V318,V286,V302,V285,V295,V280,V284,V307,V303,V317,V316,V312,V311,V310,V309,V308,V297,V306,V305,V304,card1,TransactionAmt,TransactionDT,C1,C2,C3,C4,C6,C7,C8,id_01,C10,C11,C12,C13,C14,TransactionID,id_33,id_30,id_34,DeviceInfo,id_16,P_emaildomain,R_emaildomain,id_31,DeviceType,id_29,id_28,id_38,id_35,id_37,id_15,id_36,card4,card6,ProductCD,id_12
None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,32.0,-300.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87.0,204.0,27.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,-1.0,2.0,600.0,542.0,166.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,200440.0,100.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,137.0,399.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,9305,50.0,21323525,1.0,1.0,0.0,1.0,1.0,0.0,1.0,-5.0,1.0,1.0,0.0,1.0,1.0,3750241,2048x1536,iOS 11.4.1,match_status:2,iOS Device,NotFound,aol.com,yahoo.com,mobile safari 11.0,mobile,NotFound,New,F,T,F,New,F,american express,credit,R,NotFound
None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,N

In [17]:
# get categorical and numerical columns
cat = ['id_33', 'id_30', 'id_34', 'DeviceInfo', 'id_16', 
       'P_emaildomain', 'R_emaildomain', 'id_31', 'DeviceType', 
       'id_29', 'id_28', 'id_38', 'id_35', 'id_37', 'id_15',
      'id_36', 'card4', 'card6', 'ProductCD', 'id_12'] 
num = [c for c in df_train.columns if c not in cat]

numcat = num + cat
types = ['float' for i in range(len(num))] + ['string' for i in range(len(cat))]
numcat_types = list(zip(numcat, types))
print(numcat_types[:5])
print(numcat_types[-5:])

In [19]:
# converting numbers from string to float

df_train2 = df_train.select(*(df_train[col].cast(types).alias(col)
                        for col, types in numcat_types))
display(df_train2)

D3,D2,D5,D7,dist2,id_18,D12,D13,D14,V50,V38,V40,V52,V51,V37,V47,V46,V44,V43,V42,V41,V45,V39,D4,id_04,id_03,D6,V56,V55,V57,V64,V58,V71,V60,V61,V62,V63,V72,V59,V68,V65,V74,V73,V67,V66,V92,V85,V93,V94,V88,V87,V86,V78,V84,V83,V82,V80,V79,V77,V81,V89,D15,V34,V22,V33,V14,V15,V16,V17,V18,V19,V20,V21,V23,V24,V25,V26,V27,V28,V32,V31,D10,D9,D8,id_10,id_09,id_32,id_14,V147,V148,V149,V153,V154,V155,V142,V141,V156,V157,V140,V146,V161,V139,V138,V163,V162,V158,V164,V143,V144,V145,V150,V151,V152,V159,V165,V160,V166,V331,V333,V334,V332,V335,V336,V337,V324,V325,V330,V329,V339,V328,V327,V326,V322,V323,V338,addr2,addr1,id_13,V253,V269,V274,V275,V276,V277,V278,V252,V235,V233,V232,V231,V230,V229,V228,V226,V217,V218,V225,V224,V219,V273,V223,V268,V258,V249,V248,V247,V246,V244,V243,V242,V241,V240,V237,V267,V254,V257,V236,V266,V265,V260,V264,V263,V261,V262,id_06,id_05,id_20,id_19,id_17,V203,V214,V193,V215,V216,V199,V204,V213,V205,V206,V207,V196,V202,V168,V192,V178,V211,V167,V172,V173,V176,V177,V179,V191,V181,V182,V183,V186,V187,V190,V212,V169,V210,V194,V209,V170,V171,V174,V175,V180,V184,V188,V189,V185,V195,V197,V198,V200,V201,V208,id_02,id_11,V251,V238,V239,V234,V256,V255,V227,V222,V250,V221,V220,V270,V245,V271,V272,V259,card5,card2,V296,V314,V313,D1,V300,V301,V288,V315,V281,V282,V283,V289,card3,V104,V105,V95,V96,V97,V98,V100,V101,V102,V103,V99,V116,V106,V123,V137,V136,V135,V134,V133,V132,V131,V130,V129,V128,V127,V126,V125,V124,V122,V107,V121,V120,V119,V118,V117,V115,V114,V113,V112,V111,V110,V109,V108,V320,V321,V291,V319,V279,V298,V294,V293,V292,V299,V290,V287,V318,V286,V302,V285,V295,V280,V284,V307,V303,V317,V316,V312,V311,V310,V309,V308,V297,V306,V305,V304,card1,TransactionAmt,TransactionDT,isFraud,C1,C2,C3,C4,C6,C7,C8,id_01,C10,C11,C12,C13,C14,TransactionID,id_33,id_30,id_34,DeviceInfo,id_16,P_emaildomain,R_emaildomain,id_31,DeviceType,id_29,id_28,id_38,id_35,id_37,id_15,id_36,card4,card6,ProductCD,id_12
0.0,0.0,0.0,0.0,744.0,15.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.9166659712791443,51.916664123535156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,486.0,52.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,-3.0,0.0,401.0,290.0,225.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,164420.0,100.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,224.0,375.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,185.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,4461.0,153.22500610351562,684163.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,-5.0,1.0,1.0,1.0,1.0,1.0,3014142.0,None,None,None,Windows,Found,hotmail.com,hotmail.com,chrome 62.0,desktop,Found,Found,T,F,T,Found,F,mastercard,debit,C,NotFound
305.0,305.0,305.0,305.0,0.0,15.0,305.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,305.0,0.0,0.0,305.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8333330154418945,75.83333587646484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
# converting numbers from string to float

# get categorical and numerical columns
cat = ['id_33', 'id_30', 'id_34', 'DeviceInfo', 'id_16', 
       'P_emaildomain', 'R_emaildomain', 'id_31', 'DeviceType', 
       'id_29', 'id_28', 'id_38', 'id_35', 'id_37', 'id_15',
      'id_36', 'card4', 'card6', 'ProductCD', 'id_12'] 
num = [c for c in df_test.columns if c not in cat]

numcat = num + cat
types = ['float' for i in range(len(num))] + ['string' for i in range(len(cat))]
numcat_types = list(zip(numcat, types))
print(numcat_types[:5])
print(numcat_types[-5:])

df_test2 = df_test.select(*(df_test[col].cast(types).alias(col)
                        for col, types in numcat_types))
display(df_test2)

D3,D2,D5,D7,dist2,id_18,D12,D13,D14,V50,V38,V40,V52,V51,V37,V47,V46,V44,V43,V42,V41,V45,V39,D4,id_04,id_03,D6,V56,V55,V57,V64,V58,V71,V60,V61,V62,V63,V72,V59,V68,V65,V74,V73,V67,V66,V92,V85,V93,V94,V88,V87,V86,V78,V84,V83,V82,V80,V79,V77,V81,V89,D15,V34,V22,V33,V14,V15,V16,V17,V18,V19,V20,V21,V23,V24,V25,V26,V27,V28,V32,V31,D10,D9,D8,id_10,id_09,id_32,id_14,V147,V148,V149,V153,V154,V155,V142,V141,V156,V157,V140,V146,V161,V139,V138,V163,V162,V158,V164,V143,V144,V145,V150,V151,V152,V159,V165,V160,V166,V331,V333,V334,V332,V335,V336,V337,V324,V325,V330,V329,V339,V328,V327,V326,V322,V323,V338,addr2,addr1,id_13,V253,V269,V274,V275,V276,V277,V278,V252,V235,V233,V232,V231,V230,V229,V228,V226,V217,V218,V225,V224,V219,V273,V223,V268,V258,V249,V248,V247,V246,V244,V243,V242,V241,V240,V237,V267,V254,V257,V236,V266,V265,V260,V264,V263,V261,V262,id_06,id_05,id_20,id_19,id_17,V203,V214,V193,V215,V216,V199,V204,V213,V205,V206,V207,V196,V202,V168,V192,V178,V211,V167,V172,V173,V176,V177,V179,V191,V181,V182,V183,V186,V187,V190,V212,V169,V210,V194,V209,V170,V171,V174,V175,V180,V184,V188,V189,V185,V195,V197,V198,V200,V201,V208,id_02,id_11,V251,V238,V239,V234,V256,V255,V227,V222,V250,V221,V220,V270,V245,V271,V272,V259,card5,card2,V296,V314,V313,D1,V300,V301,V288,V315,V281,V282,V283,V289,card3,V104,V105,V95,V96,V97,V98,V100,V101,V102,V103,V99,V116,V106,V123,V137,V136,V135,V134,V133,V132,V131,V130,V129,V128,V127,V126,V125,V124,V122,V107,V121,V120,V119,V118,V117,V115,V114,V113,V112,V111,V110,V109,V108,V320,V321,V291,V319,V279,V298,V294,V293,V292,V299,V290,V287,V318,V286,V302,V285,V295,V280,V284,V307,V303,V317,V316,V312,V311,V310,V309,V308,V297,V306,V305,V304,card1,TransactionAmt,TransactionDT,C1,C2,C3,C4,C6,C7,C8,id_01,C10,C11,C12,C13,C14,TransactionID,id_33,id_30,id_34,DeviceInfo,id_16,P_emaildomain,R_emaildomain,id_31,DeviceType,id_29,id_28,id_38,id_35,id_37,id_15,id_36,card4,card6,ProductCD,id_12
nan,nan,nan,nan,nan,15.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.5,349.5,0.0,0.0,nan,nan,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87.0,126.0,27.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,-1.0,10.0,549.0,352.0,166.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,205583.0,100.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,102.0,170.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,15066.0,200.0,22508556.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,-5.0,1.0,1.0,0.0,0.0,0.0,3783378.0,None,None,match_status:2,SM-G950U Build/R16NW,Found,aol.com,aol.com,chrome 68.0 for android,mobile,Found,Found,F,F,T,Found,F,mastercard,credit,R,NotFound
5.0,nan,0.0,0.0,180.0,15.0,5.0,0.0,110.0,1.0,8.0,3.0,1.0,1.0,5.0,1.0,1.0,11.0,3.0,1.0,1.0,13.0,1.0,5.0,nan,nan,5.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,13.0,11.0,8.0,1.0,1.0,1.0,1.0,1.0,5.0,3.0,0.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na

In [22]:
df_train2.createOrReplaceTempView('df_train2')
df_test2.createOrReplaceTempView('df_test2')

### Duplicates

In [23]:
query = """
SELECT COUNT(*), COUNT(DISTINCT(transactionID))
FROM df_train2
"""

sql(query).show()

+--------+-----------------------------+
|count(1)|count(DISTINCT transactionID)|
+--------+-----------------------------+
|  144233|                       144233|
+--------+-----------------------------+



In [24]:
query = """
SELECT COUNT(*), COUNT(DISTINCT(transactionID))
FROM df_test2
"""

sql(query).show()

+--------+-----------------------------+
|count(1)|count(DISTINCT transactionID)|
+--------+-----------------------------+
|  141907|                       141907|
+--------+-----------------------------+



### Null Values

In [25]:
from pyspark.sql.functions import col, sum

# replace all null values with 0
df_train3 = df_train2.fillna(value=0)

nulls = df_train3.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df_train3.columns))
nulls_pd = nulls.toPandas().transpose()

num_trans = df_train3.count()
nulls_pd.columns = ['counts']
nulls_pd["prop"] = nulls_pd.counts / num_trans
nulls_pd.sort_values(by="prop", ascending=False)

,counts,prop
id_33,70944,0.491871
id_30,66668,0.462224
id_34,66428,0.460560
DeviceInfo,25567,0.177262
id_16,14893,0.103257
P_emaildomain,13391,0.092843
R_emaildomain,13150,0.091172
id_31,3951,0.027393
DeviceType,3423,0.023732
id_28,3255,0.022568


In [26]:
# replace all null values with 0
df_test3 = df_test2.fillna(value=0)

nulls = df_test3.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df_test3.columns))
nulls_pd = nulls.toPandas().transpose()

num_trans = df_test3.count()
nulls_pd.columns = ['counts']
nulls_pd["prop"] = nulls_pd.counts / num_trans
nulls_pd.sort_values(by="prop", ascending=False)

,counts,prop
id_30,71248,0.502075
id_33,71236,0.501991
id_34,69732,0.491392
DeviceInfo,26850,0.189208
id_16,16160,0.113877
P_emaildomain,11660,0.082166
R_emaildomain,9797,0.069038
id_31,5282,0.037222
id_28,5129,0.036143
id_29,5129,0.036143


### Stats Per Numerical Column

In [ ]:
# min, max, mean, stddev

stats_train = df_train3.describe()
display(stats_train)

In [ ]:
# min, max, mean, stddev

stats_test = df_test3.describe()
display(stats_test)

In [36]:
df_test3.count(), df_train3.count()

(141907, 144233)

In [32]:
# boxplots on sample?
df_test_sample = df_test3.sample(fraction=5000, seed=0)
df_train_sample = df_train3.sample(fraction=5000, seed=0)

TypeError: withReplacement (optional), fraction (required) and seed (optional) should be a bool, float and number; however, got [<class 'int'>, <class 'int'>].

### Categorical Values

In [ ]:
# distinct values

In [ ]:
# case